# Tutorial: Julia Basics

## Computational Economics  (ECO309)

### What is Julia

- developped at MIT on top of opensource technologies
    - linux / git / llvm
- syntax inspired by Matlab but:
    - more consistent
    - lots of features from high level languages
- everything is JIT-compiled
    - interpreted vs compiled treadeoff
    - -> very fast
    - most of the base library is written in Julia
- opensource/free + vibrant community


Some useful links from QuantEcon:

* [Julia cheatsheet](https://cheatsheets.quantecon.org/julia-cheatsheet.html)
* [Julia-Matlab comparison](https://cheatsheets.quantecon.org/index.html)
* [Julia essentials](https://lectures.quantecon.org/jl/julia_essentials.html)
* [Vectors, arrays and matrices](https://lectures.quantecon.org/jl/julia_arrays.html)

Excellent resources at: [julialang](https://julialang.org/learning/)
- checkout JuliaAcademy, it's free

### an example of what you shouldn't do in Matlab

How I learnt: interpreted code is slow, so vectorize your coe.

In [3]:
function stupid_loop(I,J,K)
    t = 0.0
    for i=1:I
        for j=1:J
            for k = 1:K
                t += 1.0
            end        
        end
    end
    return t
end
@time [ stupid_loop(1000,1000,i) for i =1:10]

  0.208609 seconds (63.54 k allocations: 3.423 MiB, 41.77% compilation time)


10-element Vector{Float64}:
 1.0e6
 2.0e6
 3.0e6
 4.0e6
 5.0e6
 6.0e6
 7.0e6
 8.0e6
 9.0e6
 1.0e7

Code is translated to LLVM code then to instructions for the processor. Note that processor instructions are shorter than LLVM code.

In [4]:
@code_llvm stupid_loop(10,10,10)

;  @ c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:1 within `stupid_loop`
; Function Attrs: uwtable
define double @julia_stupid_loop_908(i64 signext %0, i64 signext %1, i64 signext %2) #0 {
top:
;  @ c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:3 within `stupid_loop`
; ┌ @ range.jl:5 within `Colon`
; │┌ @ range.jl:354 within `UnitRange`
; ││┌ @ range.jl:359 within `unitrange_last`
     %.inv = icmp sgt i64 %0, 0
     %3 = select i1 %.inv, i64 %0, i64 0
; └└└
  br i1 %.inv, label %L12.preheader, label %L76

L12.preheader:                                    ; preds = %top
  %.inv26 = icmp sgt i64 %1, 0
  %4 = select i1 %.inv26, i64 %1, i64 0
  %.inv27 = icmp sgt i64 %2, 0
  %5 = select i1 %.inv27, i64 %2, i64 0
;  @ c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:4 within `stupid_loop`
  %brmerge.demorgan = and i1 %.inv26, %.inv27
  br i1 %brmerge.demorgan, label %L25.preheader.split.us.us.us, label %L76

L64.us.us:                 

In [5]:
@code_native stupid_loop(10,10,10)

	.text
; ┌ @ 1_1_Julia_Basics.ipynb:1 within `stupid_loop`
	pushq	%rbp
	movq	%rsp, %rbp
	vxorpd	%xmm0, %xmm0, %xmm0
; │ @ 1_1_Julia_Basics.ipynb:3 within `stupid_loop`
; │┌ @ range.jl:5 within `Colon`
; ││┌ @ range.jl:354 within `UnitRange`
; │││┌ @ range.jl:359 within `unitrange_last`
	testq	%rcx, %rcx
; │└└└
	jle	L161
; │ @ 1_1_Julia_Basics.ipynb within `stupid_loop`
	testq	%rdx, %rdx
; │ @ 1_1_Julia_Basics.ipynb:4 within `stupid_loop`
	jle	L161
	testq	%r8, %r8
	jle	L161
; │ @ 1_1_Julia_Basics.ipynb within `stupid_loop`
	movq	%rdx, %r10
	movq	%rcx, %rax
	vxorpd	%xmm0, %xmm0, %xmm0
	sarq	$63, %r10
	sarq	$63, %rax
	andnq	%rdx, %r10, %r10
	movabsq	$.rodata.cst8, %rdx
	andnq	%rcx, %rax, %r9
	movq	%r8, %rcx
	vmovsd	(%rdx), %xmm1                   # xmm1 = mem[0],zero
	sarq	$63, %rcx
	andnq	%r8, %rcx, %rcx
	movl	$1, %r8d
	nop
L96:
	movl	$1, %eax
	nopw	%cs:(%rax,%rax)
L112:
	movq	%rcx, %rdx
	nopw	%cs:(%rax,%rax)
; │ @ 1_1_Julia_Basics.ipynb:6 within `stupid_loop`
; │┌ @ float.jl:399 within 

### Syntax Review

#### Variable assignment

Assignement operator is = (equality is ==, identity is ===)

In [6]:
# Assign the value 10 to the variable x
x = 10

10

In [7]:
2 == 3

false

In [8]:
# Variable names can have Unicode characters
# To get ϵ in the REPL, type \epsilon<TAB>
ϵ = 1e-4

0.0001

Default semantic is pass-by-reference:

In [10]:
a = [1,2,3,4]
b = a
a[1] = 10
b

4-element Vector{Int64}:
 10
  2
  3
  4

To work on a copy: `copy` or `deepcopy`

In [11]:
a = [1,2,3,4]
b = copy(a)
a[1]=10
b

4-element Vector{Int64}:
 1
 2
 3
 4

In [13]:
a .== b

4-element BitVector:
 0
 1
 1
 1

In [14]:
a === b

false

#### Basic types

In [15]:
# for any object `typeof` returns the type
?typeof

ErrorException: syntax: invalid identifier name "?"

In [16]:
typeof(a)

Vector{Int64} (alias for Array{Int64, 1})

#### Numbers

In [17]:
y = 2 + 2

4

In [18]:
-y

-4

In [19]:
0.34*23

7.82

In [20]:
3/4

0.75

In [21]:
# Scalar multiplication doesn't require *
3(4 - 2)

6

In [22]:
x = 4
2x

8

In [23]:
typeof(x)

Int64

In [24]:
sizeof(a)

32

#### Booleans

Equality

In [25]:
0 == 1

false

In [26]:
2 != 3

true

In [27]:
3 <= 4

true

Identity

In [28]:
a = [34, 35]
b = [34, 35]
c = a

2-element Vector{Int64}:
 34
 35

In [29]:
c === a

true

In [30]:
b === a

false

Boolean operator

In [31]:
true && false

false

In [32]:
true || false

true

In [33]:
!true

false

#### Strings

In [34]:
# Strings are written using double quotes
str = "This is a string"

"This is a string"

In [35]:
ch = 'k' # this is a character

'k': ASCII/Unicode U+006B (category Ll: Letter, lowercase)

In [36]:
# Strings can also contain Unicode characters
fancy_str = "α is a string"

"α is a string"

In [37]:
# String interpolation using $
# The expression in parentheses is evaluated and the result is 
# inserted into the string
a = 2+2
"2 + 2 = $(a)"

"2 + 2 = 4"

In [38]:
println("It took me $(a) iterations")

It took me 4 iterations


In [39]:
# String concatenation using *
"hello" * "world"

"helloworld"

In [40]:
println("hello ", "world")

hello world


#### Arrays

Julia has one-dimensional arrays. They are also called Vector.

In [44]:
A = [1, 2]

2-element Vector{Int64}:
 1
 2

In [42]:
sizeof(A)

16

In [45]:
typeof(A) == Vector{Int64}

true

In [46]:
A''

2-element Vector{Int64}:
 1
 2

In [47]:
# vectors have one dimension: they are indexed by an integer
A[1]

1

2d arrays are also called matrices... and can be used for matrix multiplications.

In [48]:
a1 = [1,2,3,4]
a2 = [1,2,3,4].+4
println( [a1; a2] )
cat(a1, a2; dims=1)

[1, 2, 3, 4, 5, 6, 7, 8]


8-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
 7
 8

In [49]:
b = [1 0.6 0]

1×3 Matrix{Float64}:
 1.0  0.6  0.0

In [50]:
B = [0.1 0.2 0.3; 4 5 6]

2×3 Matrix{Float64}:
 0.1  0.2  0.3
 4.0  5.0  6.0

In [51]:
B*B'

2×2 Matrix{Float64}:
 0.14   3.2
 3.2   77.0

Vectorized operations take a ., even comparisons:

In [52]:
B.*B

2×3 Matrix{Float64}:
  0.01   0.04   0.09
 16.0   25.0   36.0

In [53]:
B .* B .< B

2×3 BitMatrix:
 1  1  1
 0  0  0

In [54]:
f(x) = x^2+1

f (generic function with 1 method)

In [55]:
f(43)

1850

In [56]:
f.(B)

2×3 Matrix{Float64}:
  1.01   1.04   1.09
 17.0   26.0   37.0

Elements are always accessed with square brackets:

In [57]:
B[1,2]

0.2

In [58]:
B[:,1]

2-element Vector{Float64}:
 0.1
 4.0

In [59]:
B[:,1:end-1]

2×2 Matrix{Float64}:
 0.1  0.2
 4.0  5.0

#### Control flow

Conditions

In [60]:
x = -3
if x < 0
    println("x is negative")
elseif x > 0 # optional and unlimited
    println("x is positive")
else         # optional
    println("x is zero")
end

x is negative


In [61]:
print("a")
print("b")
print("c")

abc

In [62]:
println("a")
println("b")
println("c")

a
b
c


While

In [63]:
i = 3
while i > 0
    println(i)
    i = i - 1
end

3
2
1


For loops

In [64]:
# Iterate through ranges of numbers
for i = 1:3
    println(i)
    
    if i == 2
        break
    end
end

1
2


In [65]:
# Iterate through arrays
cities = ["Boston", "New York", "Philadelphia"]
for city in cities
    println(city)
end

Boston
New York
Philadelphia


In [66]:
states = ["MA", "NY", "PA"]


3-element Vector{String}:
 "MA"
 "NY"
 "PA"

In [67]:
( zip(cities, states) )

zip(["Boston", "New York", "Philadelphia"], ["MA", "NY", "PA"])

In [68]:
collect( zip(cities, states) )

3-element Vector{Tuple{String, String}}:
 ("Boston", "MA")
 ("New York", "NY")
 ("Philadelphia", "PA")

In [69]:
for t in zip(cities, states)
    println(t)
end


("Boston", "MA")
("New York", "NY")
("Philadelphia", "PA")


In [42]:
# Iterate through arrays of tuples using zip
for (city, state) in zip(cities, states)
    println("$city, $state")
end

Boston, MA
New York, NY
Philadelphia, PA


In [70]:
# Iterate through arrays and their indices using enumerate
for (i, city) in enumerate(cities)
    println("City number $i is $city")
end

City number 1 is Boston
City number 2 is New York
City number 3 is Philadelphia


#### List comprehensions

In [71]:
[i^2 for i=1:10]

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [72]:
[i^2 for i=1:10 if mod(i,2)==0]

5-element Vector{Int64}:
   4
  16
  36
  64
 100

## Function definitions

In [73]:
function f(a,b,c)
    res = a+ b +c
    return res
end

f (generic function with 2 methods)

In [74]:
g(x,y) = x^2 + y^2

g (generic function with 1 method)

In [75]:
x -> x^2 + 1 

#13 (generic function with 1 method)

### Data Types and multiple dispatch

#### Composite types

A **composite type** is a collection of named fields that can be treated as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`, constrains a field to contain values of a certain type. This is optional for any field.

In [1]:
# Type definition
struct Parameter
    value::Float64
    transformation::Function # Function is a type!
    tex_label::String
    description::String
end

When a type with $n$ fields is defined, a constructor (function that creates an instance of that type) that takes $n$ ordered arguments is automatically created. Additional constructors can be defined for convenience.

In [2]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\beta", "Discount rate")

Parameter(0.9, identity, "\beta", "Discount rate")

In [3]:
β.value

0.9

In [4]:
Parameter(value, transformation, tex) = Parameter(value, transformation, tex, "no description")

Parameter

In [5]:
Parameter(0.34, identity, "\beta")

Parameter(0.34, identity, "\beta", "no description")

In [6]:
# constructor A
Parameter(value, tex) = Parameter(value, identity, tex, "no description")

Parameter

In [7]:
Parameter(0.1, "\beta")

Parameter(0.1, identity, "\beta", "no description")

In [8]:
# constructor B
Parameter(value, transformation)  = Parameter(value, transformation, "notex", "no description")

Parameter

In [9]:
Parameter(0.1, "\beta")

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Function
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at C:\Users\tress\AppData\Local\Programs\Julia-1.7.2\share\julia\base\essentials.jl:218

In [15]:
# here Julia doesn't know whether it should call constructor A or constructor B

In [10]:
# solution : give different signatures to the various constructors
Parameter(value, tex::String) = Parameter(value, identity, tex, "no description")
Parameter(value, transformation::Function)  = Parameter(value, transformation, "notex", "no description")

Parameter

In [11]:
Parameter(0.5, "\beta")

Parameter(0.5, identity, "\beta", "no description")

In [12]:
Parameter(0.9, u->u^2)

Parameter(0.9, var"#9#10"(), "notex", "no description")

In [13]:
methods( Parameter )

# 6 methods for type constructor:
[1] Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:3
[2] Parameter(value, transformation::Function) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:3
[3] Parameter(value, tex::String) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:2
[4] Parameter(value, transformation) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:2
[5] Parameter(value, transformation, tex) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:1
[6] Parameter(value, transformation, tex_label, description) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:3

In [14]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

Parameter(0.5, identity, "\alpha", "No description available")

Now the function `Parameter` has two different `methods` with different signatures:

In [15]:
methods(Parameter)

# 7 methods for type constructor:
[1] Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:3
[2] Parameter(value, transformation::Function) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:3
[3] Parameter(value::Float64, tex_label::String) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:3
[4] Parameter(value, tex::String) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:2
[5] Parameter(value, transformation) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:2
[6] Parameter(value, transformation, tex) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:1
[7] Parameter(value, transformation, tex_label, description) in Main at c:\Users\tress\Documents\eco309\tutorials\1_1_Julia_Basics.ipynb:3

In [16]:
fun(x::Int64, y::Int64) = x^3 + y

fun (generic function with 1 method)

In [17]:
fun(x::Float64, y::Int64) = x/2 + y

fun (generic function with 2 methods)

In [18]:
fun(2, 2)

10

In [19]:
fun(2.0, 2)

3.0

In [20]:
# Find the fields of an instance of a composite type
fieldnames(α)

MethodError: MethodError: no method matching fieldnames(::Parameter)
Closest candidates are:
  fieldnames(!Matched::Core.TypeofBottom) at C:\Users\tress\AppData\Local\Programs\Julia-1.7.2\share\julia\base\reflection.jl:188
  fieldnames(!Matched::Type{<:Tuple}) at C:\Users\tress\AppData\Local\Programs\Julia-1.7.2\share\julia\base\reflection.jl:190
  fieldnames(!Matched::DataType) at C:\Users\tress\AppData\Local\Programs\Julia-1.7.2\share\julia\base\reflection.jl:185
  ...

In [21]:
α.tex_label

"\alpha"

In [22]:
# Access a particular field using .
α.value

0.5

In [ ]:
# Fields are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

In [23]:
# smallest epsilon one can represent with a given float type
ϵ = sqrt(eps(Float64))

1.4901161193847656e-8

In [24]:
a = 2 + ϵ

2.000000014901161

In [25]:
a - 2

1.4901161193847656e-8

In [26]:
f(x) = x^2

f (generic function with 1 method)

In [27]:
(f(2+ϵ) - f(2))/ϵ

4.0

In [28]:
eps(Float32)

1.1920929f-7

### Mutable vs non mutable types


by default structures in Julia are non-mutable

In [29]:
α.value = 0.6

ErrorException: setfield!: immutable struct of type Parameter cannot be changed

In [30]:
mutable struct Params
    x:: Float64
    y:: Float64
end

In [31]:
pos = Params(0.4, 0.2)

Params(0.4, 0.2)

In [32]:
pos.x = 0.5

0.5

### Parameterized Types

**Parameterized types** are data types that are defined to handle values identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [25]:
# Defining a parametric point
struct Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

In [28]:
Duple(3, -1.0)

MethodError: MethodError: no method matching Duple(::Int64, ::Float64)
Closest candidates are:
  Duple(::T, !Matched::T) where T at In[25]:3

In [29]:
struct Truple{T}
    x::Duple{T}
    z::T
end

This single declaration defines an unlimited number of new types: `Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [30]:
sizeof(3.0)

8

In [31]:
sizeof( Duple(3.0, -15.0) )

16

In [ ]:
# What happens here?
Duple(1.5, 3)

In [35]:
struct Truple3{T,S}
    x::Tuple{T,S}
    z::S
end

We can also restrict the type parameter `T`:

In [37]:
typeof("S") <: Number

false

In [38]:
typeof(4) <: Number

true

In [40]:
# T can be any subtype of Number, but nothing else
struct PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [41]:
PlanarCoordinate("4th Ave", "14th St")

MethodError: MethodError: no method matching PlanarCoordinate(::String, ::String)

In [42]:
PlanarCoordinate(2//3, 8//9)

PlanarCoordinate{Rational{Int64}}(2//3, 8//9)

### Arrays are an exemple of mutable, parameterized types

### Why Use Types?

You can write all your code without thinking about types at all. If you do this, however, you’ll be missing out on some of the biggest benefits of using Julia.

If you understand types, you can:

- Write faster code
- Write expressive, clear, and well-structured programs (keep this in mind when we talk about functions)
- Reason more clearly about how your code works

Even if you only use built-in functions and types, your code still takes advantage of Julia’s type system. That’s why it’s important to understand what types are and how to use them.

In [49]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Integer)  
    sum = 0:: Integer
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end  

function sumofsins_stable(n::Integer)  
    sum = 0.0 :: Float64
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end

# Compile and run
sumofsins_unstable(Int(1e5))
sumofsins_stable(Int(1e5))

-25554.110202663698

In [50]:
@time sumofsins_unstable(Int(1e5))

  0.000176 seconds


-25554.110202663698

In [47]:
@time sumofsins_stable(Int(1e5))

  0.000168 seconds


-25554.110202663698

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type `Float64` throughout; therefore, it saves time and memory. On the other hand, in `sumofsins_unstable`, the compiler must check the type of `sum` at each iteration of the loop. Let's look at the LLVM [intermediate representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

### Multiple Dispatch

So far we have defined functions over argument lists of any type. Methods allow us to define functions “piecewise”. For any set of input arguments, we can define a **method**, a definition of one possible behavior for a function.

In [51]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

print_type (generic function with 1 method)

In [53]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

print_type (generic function with 2 methods)

In [54]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

print_type (generic function with 3 methods)

In [55]:
# See all methods for a given function
methods(print_type)

# 3 methods for generic function "print_type":
[1] print_type(x::String) in Main at In[53]:3
[2] print_type(x::Number) in Main at In[51]:3
[3] print_type(x::Number, y::Number) in Main at In[54]:3

Julia uses **multiple dispatch** to decide which **method** of a **function** to execute when a function is applied. In particular, Julia compares the types of _all_ arguments to the signatures of the function’s methods in order to choose the applicable one, not just the first (hence "multiple").

In [56]:
print_type(5)

5 is a number


In [57]:
print_type("foo")

foo is a string


In [58]:
print_type([1, 2, 3])

MethodError: MethodError: no method matching print_type(::Array{Int64,1})
Closest candidates are:
  print_type(!Matched::String) at In[53]:3
  print_type(!Matched::Number) at In[51]:3
  print_type(!Matched::Number, !Matched::Number) at In[54]:3

#### Other types of functions

Julia supports a short function definition for one-liners

In [ ]:
f(x::Float64) = x^2.0
f(x::Int64) = x^3

As well as a special syntax for anonymous functions

In [ ]:
u->u^2

In [ ]:
map(u->u^2, [1,2,3,4])

### Keyword arguments and optional arguments

In [59]:
f(a,b,c=true; algo="newton")

UndefVarError: UndefVarError: f not defined

### Packing/unpacking


In [60]:
t = (1,2,4)

(1, 2, 4)

In [61]:
a,b,c = t

(1, 2, 4)

In [65]:
[(1:10)...]

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [66]:
cat([4,3], [0,1]; dims=1)

4-element Array{Int64,1}:
 4
 3
 0
 1

In [67]:
l = [[4,3], [0,1], [0, 0], [1, 1]]
# how do I concatenate it ?

cat(l...; dims=1) ### see python's f(*s)

8-element Array{Int64,1}:
 4
 3
 0
 1
 0
 0
 1
 1

### Writing Julian Code

As we've seen, you can use Julia just like you use MATLAB and get faster code. However, to write faster and _better_ code, attempt to write in a “Julian” manner:

- Define composite types as logically needed
- Write type-stable functions for best performance
- Take advantage of multiple dispatch to write code that looks like math
- Add methods to existing functions

### Just-in-Time Compilation

How is Julia so fast? Julia is just-in-time (JIT) compiled, which  means (according to [this StackExchange answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)):

> A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. _A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently._

> This is in contrast to a traditional compiler that compiles all the code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how your code is compiled. This is why writing type-stable code makes such a difference in speed!

## Additional Exercises

Taken from QuantEcon's [Julia Essentials](https://lectures.quantecon.org/jl/julia_essentials.html) and [Vectors, Arrays, and Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lectures.

1. Consider the polynomial $$p(x) = \sum_{i=0}^n a_0 x^0$$ Using `enumerate`, write a function `p` such that `p(x, coeff)` computes the value of the polynomial with coefficients `coeff` evaluated at `x`.

2. Write a function `solve_discrete_lyapunov` that solves the discrete Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative procedure $$S_0 = \Sigma \Sigma'$$ $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a number of iterations.